In [2]:
import params
import wandb

import pandas as pd
import os
import numpy as np

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import recall_score

In [3]:
path = '/home/gianfree/Desktop/CISI-project-MLOps/'

def read_mappings():
    rel_set = {}
    with open(os.path.join(path, 'CISI.REL')) as f:
        for l in f.readlines():
            qry_id = int(l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0])
            doc_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1]

            if qry_id in rel_set:
                rel_set[qry_id].append(doc_id)
            else:
                rel_set[qry_id] = []
                rel_set[qry_id].append(doc_id)
    return rel_set

In [4]:
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="baseline")

wandb: Currently logged in as: gianfranco-romani. Use `wandb login --relogin` to force relogin


In [5]:
split_dataset = run.use_artifact('gianfranco-romani/cisi-project/data-splits:latest', type='dataset')
train = split_dataset.get("train-data")
test = split_dataset.get("test-data")
val = split_dataset.get("val-data")

wandb:   3 of 3 files downloaded.  
wandb:   3 of 3 files downloaded.  
wandb:   3 of 3 files downloaded.  


In [6]:
mappings = read_mappings()

In [7]:
model = SentenceTransformer('all-MiniLM-L6-v2')
emb_queries = model.encode(test.get_column("query")[:10])
emb_questions = model.encode(test.get_column("doc_text")[:10])

In [ ]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

evaluator = EmbeddingSimilarityEvaluator.from_input_examples(sts_reader.get_examples('sts-dev.csv'))

In [8]:
res = cosine_similarity(emb_queries, emb_questions)
#res

array([[ 0.2847346 ,  0.25719053,  0.10240063,  0.12293756,  0.16833484,
         0.2874623 ,  0.20452464,  0.51545334,  0.19411021,  0.20697345],
       [ 0.44725487,  0.56345516,  0.360586  ,  0.2742477 ,  0.26918757,
         0.5911861 ,  0.24796732,  0.3936075 ,  0.25651917,  0.2850684 ],
       [ 0.29465538,  0.30009323,  0.3981277 ,  0.24446055,  0.13022794,
         0.41670698,  0.18812767,  0.55007005,  0.18714547,  0.25568163],
       [ 0.59377563,  0.42029706,  0.35294735,  0.35484943,  0.35322714,
         0.51941013,  0.41250026,  0.5746869 ,  0.26772305,  0.38878733],
       [ 0.59377563,  0.42029706,  0.35294735,  0.35484943,  0.35322714,
         0.51941013,  0.41250026,  0.5746869 ,  0.26772305,  0.38878733],
       [ 0.25927836,  0.2602811 ,  0.21599223,  0.01419196, -0.01462908,
         0.3048433 ,  0.10882621,  0.36733514,  0.2240171 ,  0.00804543],
       [ 0.29026145,  0.28047854,  0.28018677,  0.36115855,  0.362948  ,
         0.4385658 ,  0.3992448 ,  0.6781293 

In [38]:
#print(test.get_column("query")[0], "\n\n",test.get_column("doc_text")[0], "\n\n",test.get_column("doc_text")[7])

In [59]:
def build_result_and_test_y(res, mappings):
    #keep just the documents with similarity > 0.5 (and get their index)
    retrieved_docs = []
    for i, l in enumerate(res):
        retrieved_docs.append([])
        for j, v in enumerate(l):
            if v>=0.5:
                retrieved_docs[-1].append(j)
    retrieved_docs
    
    ids = list(mappings.keys())
    ids.sort()
    
    y_true = []
    for i in ids:
        y_true.append(mappings[i])
    return retrieved_docs, y_true

In [50]:
def recall(y_true, y_pred):
    recall_res = 0
    for i,l in enumerate(y_pred):
        if len(y_pred[i])==0:
            continue
        count = 0
        for j,v in enumerate(l):
            if v in y_true[i]:
                count+=1
        rec = count/len(y_true[i])
        recall_res += rec
    return recall_res/len(y_true)

In [63]:
def precision(y_true, y_pred):
    precision_res = 0
    for i,l in enumerate(y_pred):
        if len(y_pred[i])==0:
            continue
        count = 0
        for j,v in enumerate(l):
            if v in y_true[i]:
                count+=1
        rec = count/len(y_pred[i])
        precision_res += rec
    return precision_res/len(y_true)

In [60]:
y_pred, y_true = build_result_and_test_y(res, mappings)

In [62]:
recall_result = recall(y_true, y_pred)
recall_result

0.0

In [64]:
precision_result = precision(y_true, y_pred)
precision_result

0.0